In [142]:
from selenium import webdriver
from selenium.webdriver.common.by import By  # Importação correta do By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager  # Importação do ChromeDriverManager
import time
import os
import pandas as pd
import numpy as np

servico = Service(ChromeDriverManager().install())

# Configuração do diretório de download
download_directory = os.path.join(os.getcwd(), "dados")
if not os.path.exists(download_directory):  # Cria a pasta se não existir
    os.makedirs(download_directory)

# Opções do Chrome
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_directory,  # Define a pasta de download
    "download.prompt_for_download": False,  # Não solicita confirmação para downloads
    "directory_upgrade": True,
})

In [49]:
# Dicionário com as marcas e seus valores
dados = {
    "Marca": [
        "Itaú", "Banco do Brasil", "Bradesco", "Petrobras", "Vale", 
        "Caixa", "Localiza", "Brahma", "Natura", "Skol", 
        "Sadia", "Vivo", "Rede D’or São Luiz", "Nubank", "Marfrig", 
        "Latam Airlines", "Assai Atacadista", "Raízen", "Magazine Luiza", "Antarctica", 
        "Suzano", "Ipiranga", "Embraer", "Braskem", "Atacadão", 
        "Unidas", "Sabesp", "Klabin", "Movida", "PagSeguro Digital", 
        "Renner", "Camil", "Votorantim", "Globo", "Vibra", 
        "XP Inc.", "BNDES", "Gerdau", "Seguradora Porto", "Itambé", 
        "SmartFit", "Minerva", "Stone", "Perdigão", "Neoenergia", 
        "Hapvida", "B3", "Sul América", "Ypióca", "Banco Pan"
    ],
    "Valor": [
        8330000000, 5450000000, 5010000000, 3400000000, 3000000000, 
        2860000000, 2300000000, 1960000000, 1950000000, 1900000000, 
        1800000000, 1700000000, 1370000000, 1350000000, 1350000000, 
        1220000000, 1200000000, 1090000000, 994000000, 992000000, 
        920000000, 918000000, 864000000, 827000000, 820000000, 
        766000000, 751000000, 693000000, 682000000, 630000000, 
        615000000, 574000000, 569000000, 558000000, 553000000, 
        544000000, 543000000, 536000000, 500000000, 444000000, 
        432000000, 426000000, 416000000, 390000000, 388000000, 
        377000000, 358000000, 357000000, 354000000, 342000000
    ]
}

In [ ]:
driver = webdriver.Chrome(service=servico, options=options)

# Acessa o Google Trends
driver.get("https://trends.google.com/trends/explore?geo=BR&hl=pt-BR")

# Espera alguns segundos para garantir o carregamento da página
time.sleep(2)

# Recarrega a página
driver.refresh()

# Espera mais alguns segundos após a recarga
time.sleep(2)

# Inicializa ActionChains
actions = ActionChains(driver)

# Dicionário de XPaths dos botões
buttons = {
    "Tempo": '/html/body/div[3]/div[2]/div/md-content/div/div/div[1]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]/i',
    "Regiao": '/html/body/div[3]/div[2]/div/md-content/div/div/div[2]/trends-widget/ng-include/widget/div/div/div/widget-actions/div/button[1]/i'
}

def click_button(button_xpath):
    """
    Clica em um botão baseado no XPath fornecido, rolando a página até o botão.
    
    :param button_xpath: O XPath do botão a ser clicado.
    """
    button = driver.find_element(By.XPATH, button_xpath)
    driver.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'center' });", button)
    time.sleep(1)
    actions.move_to_element(button).click().perform()

# Percorre cada marca no DataFrame
for marca in dados["Marca"]:
    print(f"Pesquisando por: {marca}")

    # Volta para o topo da página
    driver.execute_script("window.scrollTo(0, 0);")

    # Pesquisa a marca
    search_box_xpath = '//*[@id="input-29"]'  # XPath da caixa de busca
    search_box = driver.find_element(By.XPATH, search_box_xpath)

    # Limpa a caixa de busca usando BACKSPACE
    search_box_value = search_box.get_attribute('value')  # Obtém o valor atual da caixa
    for _ in range(len(search_box_value)):
        search_box.send_keys(Keys.BACKSPACE)  # Remove um caractere por vez
    
    # Digita a nova marca e submete a busca
    search_box.send_keys(marca)
    search_box.send_keys(Keys.RETURN)

    # Espera o carregamento dos dados
    time.sleep(3)
    
    # Itera sobre os botões e clica em cada um
    for name, xpath in buttons.items():
        click_button(xpath)  # Clica no botão

        # Aguarda o download
        print(f"Aguardando o download de {name} para {marca}...")
        time.sleep(3)  # Aumentando o tempo de espera para garantir o download

    # Espera até que ambos os arquivos tenham sido baixados
    multi_timeline_path = os.path.join(download_directory, 'multiTimeline.csv')
    geo_map_path = os.path.join(download_directory, 'geoMap.csv')

    while not (os.path.exists(multi_timeline_path) and os.path.exists(geo_map_path)):
        time.sleep(1)  # Aguarda um segundo até que ambos os arquivos sejam encontrados

    # Renomeia os arquivos
    new_multi_timeline_path = os.path.join(download_directory, f"{marca}_Tempo.csv")
    new_geo_map_path = os.path.join(download_directory, f"{marca}_Regiao.csv")

    if not os.path.exists(new_multi_timeline_path):
        os.rename(multi_timeline_path, new_multi_timeline_path)  # Renomeia o arquivo multiTimeline

    if not os.path.exists(new_geo_map_path):
        os.rename(geo_map_path, new_geo_map_path)  # Renomeia o arquivo geoMap

    print(f"Download e renomeação concluídos para: {marca} - Tempo e Regiao")

    # Aguarda um pouco antes de começar a nova pesquisa
    time.sleep(2)

# Fecha o navegador ao final
driver.quit()

In [ ]:
# Lista para armazenar os DataFrames
dfs_Tempo = pd.DataFrame()
dfs_Tempo["Marca"] = dados["Marca"]

df_Tempo = pd.read_csv("dados/Skol_Tempo.csv", skiprows=1)

# Obtém as datas da coluna 'Semana'
datas = df_Tempo["Semana"].tolist()

# Cria novas colunas em dfs para cada data
for data in datas:
    coluna_nome = f"{data}"
    dfs_Tempo[coluna_nome] = np.nan  # Preenche com NaN

# Exemplo de como o diretório de arquivos CSV pode ser configurado
download_directory = "dados"

# Itera sobre cada marca no DataFrame
for marca in dfs_Tempo["Marca"]:
    print(f"Processando a marca: {marca}")

    # Cria o nome do arquivo CSV correspondente
    arquivo_csv = os.path.join(download_directory, f"{marca}_Tempo.csv")
    
    # Verifica se o arquivo existe
    if os.path.exists(arquivo_csv):
        # Lê o arquivo CSV
        df_temp = pd.read_csv(arquivo_csv, skiprows=1)
        
        # Itera sobre cada data na coluna 'Semana' de dfs
        for data in dfs_Tempo.columns[1:]:  # Ignora a primeira coluna 'Marca'
            # Nome da coluna correspondente no df_temp
            coluna_nome_temp = f"{marca}: (Brasil)"  # O formato do nome da coluna

            # Verifica se a coluna existe no df_temp
            if coluna_nome_temp in df_temp.columns:
                # Obtém o valor correspondente da coluna de df_temp
                valor = df_temp.loc[df_temp["Semana"] == data, coluna_nome_temp].values
                
                if valor.size > 0:  # Verifica se encontrou algum valor
                    # Preenche o valor na coluna correspondente em dfs
                    dfs_Tempo.loc[dfs_Tempo["Marca"] == marca, data] = valor[0]  # Atribui o valor

In [326]:
# Transformar a primeira coluna em índice para dfs_Tempo
dfs_Tempo.set_index(dfs_Tempo.columns[0], inplace=True)
dfs_Tempo.iloc[:, 1:] = dfs_Tempo.iloc[:, 1:].astype(int)

In [328]:
dfs_Tempo

,2023-10-22,2023-10-29,2023-11-05,2023-11-12,2023-11-19,2023-11-26,2023-12-03,2023-12-10,2023-12-17,2023-12-24,...,2024-08-11,2024-08-18,2024-08-25,2024-09-01,2024-09-08,2024-09-15,2024-09-22,2024-09-29,2024-10-06,2024-10-13
2023-10-15,,,,,,,,,,,,,,,,,,,,,
88.0,68.0,80.0,89.0,80.0,71.0,83.0,99.0,97.0,86.0,87.0,...,76.0,78.0,76.0,94.0,82.0,76.0,78.0,89.0,83.0,80.0
76.0,72.0,71.0,88.0,70.0,72.0,77.0,84.0,85.0,81.0,79.0,...,78.0,77.0,76.0,84.0,85.0,79.0,75.0,79.0,76.0,75.0
74.0,77.0,68.0,77.0,64.0,68.0,87.0,77.0,84.0,71.0,66.0,...,70.0,69.0,71.0,76.0,71.0,67.0,65.0,74.0,67.0,62.0
26.0,29.0,21.0,26.0,21.0,22.0,20.0,20.0,21.0,21.0,100.0,...,24.0,25.0,23.0,23.0,21.0,20.0,22.0,23.0,22.0,22.0
88.0,84.0,86.0,86.0,92.0,85.0,86.0,85.0,94.0,92.0,97.0,...,82.0,83.0,86.0,88.0,86.0,85.0,83.0,84.0,86.0,92.0
71.0,68.0,68.0,74.0,70.0,75.0,70.0,73.0,79.0,75.0,82.0,...,68.0,67.0,65.0,67.0,74.0,67.0,66.0,68.0,65.0,61.0
78.0,77.0,77.0,80.0,87.0,81.0,81.0,93.0,92.0,96.0,100.0,...,73.0,72.0,71.0,77.0,78.0,73.0,74.0,73.0,71.0,80.0
40.0,43.0,44.0,46.0,52.0,50.0,51.0,62.0,69.0,66.0,62.0,...,42.0,41.0,37.0,39.0,39.0,36.0,36.0,37.0,33.0,41.0
67.0,66.0,65.0,73.0,75.0,88.0,81.0,92.0,99.0,100.0,77.0,...,65.0,61.0,63.0,65.0,65.0,68.0,64.0,62.0,59.0,65.0


In [320]:
# Lista para armazenar os DataFrames
dfs_Regiao = pd.DataFrame()
dfs_Regiao["Marca"] = dados["Marca"]

df_Regiao = pd.read_csv("dados/Skol_Regiao.csv", skiprows=1)

# Obtém as datas da coluna 'Semana'
datas = df_Regiao["Região"].tolist()

# Cria novas colunas em dfs para cada data
for data in datas:
    coluna_nome = f"{data}"
    dfs_Regiao[coluna_nome] = np.nan  # Preenche com NaN

# Exemplo de como o diretório de arquivos CSV pode ser configurado
download_directory = "dados"

# Itera sobre cada marca no DataFrame
for marca in dfs_Regiao["Marca"]:
    print(f"Processando a marca: {marca}")

    # Cria o nome do arquivo CSV correspondente
    arquivo_csv = os.path.join(download_directory, f"{marca}_Regiao.csv")
    
    # Verifica se o arquivo existe
    if os.path.exists(arquivo_csv):
        # Lê o arquivo CSV
        df_regiao = pd.read_csv(arquivo_csv, skiprows=1)
        
        # Itera sobre cada data na coluna 'Semana' de dfs
        for data in dfs_Regiao.columns[1:]:  # Ignora a primeira coluna 'Marca'
            # Nome da coluna correspondente no df_temp
            coluna_nome_regiao = f"{marca}: (15/10/2023 – 15/10/2024)"  # O formato do nome da coluna

            # Verifica se a coluna existe no df_temp
            if coluna_nome_regiao in df_regiao.columns:
                # Obtém o valor correspondente da coluna de df_temp
                valor = df_regiao.loc[df_regiao["Região"] == data, coluna_nome_regiao].values
                
                if valor.size > 0:  # Verifica se encontrou algum valor
                    # Preenche o valor na coluna correspondente em dfs
                    dfs_Regiao.loc[dfs_Regiao["Marca"] == marca, data] = valor[0]  # Atribui o valor

FileNotFoundError: [Errno 2] No such file or directory: 'dados/Skol_Regiao.csv'

In [312]:
# Substitui valores nulos por 0
dfs_Regiao.fillna(0, inplace=True)

# Converte "<1" em NaN e, em seguida, para 0
dfs_Regiao.replace("<1", np.nan, inplace=True)

# Converte todas as colunas, exceto a primeira, para inteiro
dfs_Regiao.iloc[:, 1:] = dfs_Regiao.iloc[:, 1:].astype(int)

# Transformar a primeira coluna em índice para dfs_Regiao
dfs_Regiao.set_index(dfs_Regiao.columns[0], inplace=True)
dfs_Regiao.drop(dfs_Regiao.index[0], inplace=True)

In [314]:
dfs_Regiao

,Pará,Amapá,Maranhão,Rondônia,Roraima,Sergipe,Ceará,Tocantins,Mato Grosso,Acre,...,Mato Grosso do Sul,Amazonas,Espírito Santo,Pernambuco,Rio de Janeiro,Goiás,Distrito Federal,Paraná,Rio Grande do Sul,Santa Catarina
Piauí,,,,,,,,,,,,,,,,,,,,,
91,60,69,81,75,77,62,60,100,86,75.0,...,86,42,49,56,47,64,92,57,58,61
33,46,29,43,43,31,37,60,55,53,29.0,...,55,78,37,54,100,50,50,46,34,40
17,20,19,14,20,15,68,25,17,15,17.0,...,17,27,64,31,100,18,38,29,27,23
66,87,55,86,89,57,61,73,61,67,56.0,...,63,60,96,77,81,71,77,80,90,91
89,87,79,81,83,76,93,82,87,82,70.0,...,84,75,85,82,85,88,100,83,82,82
57,63,57,66,53,57,42,49,67,76,42.0,...,71,47,85,52,82,79,64,70,72,76
96,84,75,83,65,73,83,90,72,52,58.0,...,56,74,67,100,68,62,60,52,59,50
100,89,87,75,74,74,71,66,65,63,60.0,...,53,49,48,47,41,41,37,36,33,31
50,39,40,29,38,59,68,59,44,62,32.0,...,51,63,61,75,96,51,54,64,48,65


In [298]:
# Apagar todos os arquivos CSV no diretório
for filename in os.listdir(download_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(download_directory, filename)
        os.remove(file_path)
        print(f"Arquivo removido: {file_path}")

# Salvar os DataFrames dfs_Regiao e dfs_Tempo como arquivos CSV
dfs_Regiao.to_csv(os.path.join(download_directory, "Marcas_Regiao.csv"), index=False)
print("DataFrame dfs_Regiao salvo como 'Marcas_Regiao.csv'.")

dfs_Tempo.to_csv(os.path.join(download_directory, "Marcas_Tempo.csv"), index=False)
print("DataFrame dfs_Tempo salvo como 'Marcas_Tempo.csv'.")

Arquivo removido: dados\Marcas_Regiao.csv
Arquivo removido: dados\Marcas_Tempo.csv
DataFrame dfs_Regiao salvo como 'Marcas_Regiao.csv'.
DataFrame dfs_Tempo salvo como 'Marcas_Tempo.csv'.
